# Exporting ATT&CK Group Navigator Layers

## Get Relationship STIX Objects - (Manual)
-----------------------
I believe it is important to understand the code behind the main functions available in the Python library [attackcti](https://attackcti.readthedocs.io/en/latest/index.html). I highly recommend to first read the docs I put together about [cti-taxii-client](https://attackcti.readthedocs.io/en/latest/taxii_client.html) and [cti-python-stix2](https://attackcti.readthedocs.io/en/latest/stix.html) libraries.
Those two summarize several of the concepts that I had to read to understand how to perform a simple query against ATT&CK's TAXII server

## Import STIX and TAXII Libraries

In [3]:
from stix2 import TAXIICollectionSource, Filter, CompositeDataSource
from taxii2client.v21 import Collection

## Set ATT&CK TAXII Collection ID Variables
The public ATT&CK TAXII instance has three main collections (Enterprise, Pre and Mobile). Every collection has an ID which attackcti uses to retrieve ATT&CK STIX objects from all those matrices.

In [1]:
ATTACK_STIX_COLLECTIONS = "https://attack-taxii.mitre.org/api/v21/collections/"
ENTERPRISE_ATTACK = "x-mitre-collection--1f5f1533-f617-4ca8-9ab4-6a02367fa019"
MOBILE_ATTACK = "x-mitre-collection--dac0d2d7-8653-445c-9bff-82f934c1e858"
ICS_ATTACK = "x-mitre-collection--90c00720-636b-4485-b342-8751d232bf09"

## Initialize TAXII Collection Sources
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), the [TAXIICollectionSource API](https://stix2.readthedocs.io/en/latest/api/datastore/stix2.datastore.taxii.html#stix2.datastore.taxii.TAXIICollectionSource) provides an interface for searching/retrieving STIX objects from a local/remote TAXII Collection endpoint. In our case, we are pointing to our ATT&CK TAXII Collection instances (https://cti-taxii.mitre.org/stix/collections/<Collection ID>)

In [4]:
ENTERPRISE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ENTERPRISE_ATTACK + "/")
TC_ENTERPRISE_SOURCE = TAXIICollectionSource(ENTERPRISE_COLLECTION)
MOBILE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + MOBILE_ATTACK + "/")
TC_MOBILE_SOURCE = TAXIICollectionSource(MOBILE_COLLECTION)
ICS_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ICS_ATTACK + "/")
TC_ICS_SOURCE = TAXIICollectionSource(ICS_COLLECTION)

## Initialize a Composite Data Source
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), a user can have a single [CompositeDataSource](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource) as an interface to a set of DataSources. When an API call is made to the CompositeDataSource, it is delegated to each of the (real) DataSources that are attached to it. In our case, we have three TAXIICollection sources (Enterprise, PRE and Mobile) as defined in our previous step. Therefore, we can use the CompositeDataSource class and the add_data_sources method to attach every ATT&CK TAXIICollection source and be able to query all of them at the same time.

In [5]:
COMPOSITE_DS = CompositeDataSource()
COMPOSITE_DS.add_data_sources([TC_ENTERPRISE_SOURCE, TC_MOBILE_SOURCE, TC_ICS_SOURCE])

## Retrieve all relationships
Now that we can query all the ATT&CK TAXIICollection sources at once, we can use the query method and a set of filters to retrieve STIX objects of type relationship

In [6]:
rels = COMPOSITE_DS.query(Filter("type", "=", "relationship"))
rels[0]

Relationship(type='relationship', spec_version='2.1', id='relationship--0005fb3b-274a-4ac1-8fb2-51366fcd1a6b', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2023-09-01T21:33:59.394Z', modified='2023-09-01T21:33:59.394Z', relationship_type='mitigates', description='Consider blocking download/transfer and execution of potentially uncommon file types known to be used in adversary campaigns.', source_ref='course-of-action--21da4fd4-27ad-4e9c-b93d-0b9b14d02c96', target_ref='attack-pattern--43c9bc06-715b-42db-972f-52d25c09a20c', revoked=False, object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'], x_mitre_attack_spec_version='3.1.0', x_mitre_deprecated=False, x_mitre_modified_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', x_mitre_version='0.1')

## Retrieve all relationships from an specific STIX object
What if you want to be very specific and get relationships from a specific STIX objects? You can use the [relationships](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource.relationships) method from the [CompositeDataSource](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource) class to retrieve relationships involving a given STIX object.

In [7]:
from attackcti import attack_client
lift = attack_client()

groups = lift.get_groups()

rels = COMPOSITE_DS.relationships(groups[0], 'uses', source_only=True)
rels[0]

Relationship(type='relationship', spec_version='2.1', id='relationship--83cfa11e-f221-4dc4-b184-943c2c7f4562', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2017-05-31T21:33:27.032Z', modified='2020-02-05T14:58:20.740Z', relationship_type='uses', description='[Moafee](https://attack.mitre.org/groups/G0002) has been known to employ binary padding.(Citation: Haq 2014)', source_ref='intrusion-set--2e5d3a83-fe00-41a5-9b60-237efc84832f', target_ref='attack-pattern--5bfccc3f-2326-4112-86cc-c1ece9d8a2b5', revoked=False, external_references=[ExternalReference(source_name='Haq 2014', description='Haq, T., Moran, N., Scott, M., & Vashisht, S. O. (2014, September 10). The Path to Mass-Producing Cyber Attacks &#91;Blog&#93;. Retrieved November 12, 2014.', url='https://www.fireeye.com/blog/threat-research/2014/09/the-path-to-mass-producing-cyber-attacks.html')], object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'], x_mitre_attack_spec_version=

# Get Relationship STIX Objects - (Automatic)
-----------------------

## Retrieve all relationships

In [8]:
from attackcti import attack_client
lift = attack_client()

In [9]:
%time all_relationships = lift.get_relationships()

CPU times: user 2.41 s, sys: 140 ms, total: 2.55 s
Wall time: 21.1 s


In [10]:
all_relationships[0]

Relationship(type='relationship', spec_version='2.1', id='relationship--0005fb3b-274a-4ac1-8fb2-51366fcd1a6b', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2023-09-01T21:33:59.394Z', modified='2023-09-01T21:33:59.394Z', relationship_type='mitigates', description='Consider blocking download/transfer and execution of potentially uncommon file types known to be used in adversary campaigns.', source_ref='course-of-action--21da4fd4-27ad-4e9c-b93d-0b9b14d02c96', target_ref='attack-pattern--43c9bc06-715b-42db-972f-52d25c09a20c', revoked=False, object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'], x_mitre_attack_spec_version='3.1.0', x_mitre_deprecated=False, x_mitre_modified_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', x_mitre_version='0.1')

## Retrieve all relationships from an specific STIX object

In [11]:
groups = lift.get_groups()

In [12]:
%time group_relationships = lift.get_relationships_by_object(groups[0])

CPU times: user 607 ms, sys: 186 ms, total: 793 ms
Wall time: 37.8 s


In [13]:
group_relationships[0]

Relationship(type='relationship', spec_version='2.1', id='relationship--83cfa11e-f221-4dc4-b184-943c2c7f4562', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2017-05-31T21:33:27.032Z', modified='2020-02-05T14:58:20.740Z', relationship_type='uses', description='[Moafee](https://attack.mitre.org/groups/G0002) has been known to employ binary padding.(Citation: Haq 2014)', source_ref='intrusion-set--2e5d3a83-fe00-41a5-9b60-237efc84832f', target_ref='attack-pattern--5bfccc3f-2326-4112-86cc-c1ece9d8a2b5', revoked=False, external_references=[ExternalReference(source_name='Haq 2014', description='Haq, T., Moran, N., Scott, M., & Vashisht, S. O. (2014, September 10). The Path to Mass-Producing Cyber Attacks &#91;Blog&#93;. Retrieved November 12, 2014.', url='https://www.fireeye.com/blog/threat-research/2014/09/the-path-to-mass-producing-cyber-attacks.html')], object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'], x_mitre_attack_spec_version=

# Retrive Techniques used by one Group - (Manual)
-----------------------

In this case we want relationship objects that have target_ref values of type attack-pattern. Following the manual code I shared above, and the results from the `get_relationships_by_object()` function, you can simply query the ATT&CK Enterprise TAXIICollection source with the filter below

In [14]:
filter_objects = [
  Filter('type', '=', 'attack-pattern'),
  Filter('id', '=', [r.target_ref for r in group_relationships])
]
techniques_used = TC_ENTERPRISE_SOURCE.query(filter_objects)
techniques_used[0]

AttackPattern(type='attack-pattern', spec_version='2.1', id='attack-pattern--5bfccc3f-2326-4112-86cc-c1ece9d8a2b5', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-02-05T14:04:25.865Z', modified='2023-03-30T21:01:53.857Z', name='Binary Padding', description='Adversaries may use binary padding to add junk data and change the on-disk representation of malware. This can be done without affecting the functionality or behavior of a binary, but can increase the size of the binary beyond what some security tools are capable of handling due to file size limitations. \n\nBinary padding effectively changes the checksum of the file and can also be used to avoid hash-based blocklists and static anti-virus signatures.(Citation: ESET OceanLotus) The padding used is commonly generated by a function to create junk data and then appended to the end or applied to sections of malware.(Citation: Securelist Malware Tricks April 2017) Increasing the file size may decrease the 

# Retrive Techniques used by one Group - (Automatic)
-----------------------

In [15]:
from attackcti import attack_client
lift = attack_client()
groups = lift.get_groups()
group_techniques = lift.get_techniques_used_by_group(groups[0])
group_techniques[0]

AttackPattern(type='attack-pattern', spec_version='2.1', id='attack-pattern--5bfccc3f-2326-4112-86cc-c1ece9d8a2b5', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-02-05T14:04:25.865Z', modified='2023-03-30T21:01:53.857Z', name='Binary Padding', description='Adversaries may use binary padding to add junk data and change the on-disk representation of malware. This can be done without affecting the functionality or behavior of a binary, but can increase the size of the binary beyond what some security tools are capable of handling due to file size limitations. \n\nBinary padding effectively changes the checksum of the file and can also be used to avoid hash-based blocklists and static anti-virus signatures.(Citation: ESET OceanLotus) The padding used is commonly generated by a function to create junk data and then appended to the end or applied to sections of malware.(Citation: Securelist Malware Tricks April 2017) Increasing the file size may decrease the 

# Retrive all Techniques used by all Groups - (Manual)
-----------------------
You can apply the same get_techniques_used_by_group() function, but against all the groups STIX objects that the get_groups() function retrieves. You can do a simple for loop over more than 90 groups. However, it takes longer than what I would like it to take. Therefore, I decided to go a different route and started testing some code.

## Get all groups and techniques

In [16]:
from attackcti import attack_client
lift = attack_client()
groups = lift.get_groups()
techniques = lift.get_techniques()

## Filter Group objects using techniques

In [17]:
from stix2.utils import get_type_from_id

group_relationships = []
for rel in all_relationships:
    if get_type_from_id(rel.source_ref) == 'intrusion-set'\
    and get_type_from_id(rel.target_ref) == 'attack-pattern':
        group_relationships.append(rel)
len(group_relationships)
print(group_relationships[0])

{"type": "relationship", "spec_version": "2.1", "id": "relationship--0068ee65-0945-4f69-ba81-163ffbc05e53", "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5", "created": "2019-05-24T17:02:44.371Z", "modified": "2020-03-31T14:49:39.188Z", "relationship_type": "uses", "description": "[WIRTE](https://attack.mitre.org/groups/G0090) has used PowerShell for script execution.(Citation: Lab52 WIRTE Apr 2019)", "source_ref": "intrusion-set--f8cb7b36-62ef-4488-8a6d-a7033e3271c1", "target_ref": "attack-pattern--970a3432-3237-47ad-bcca-7d8cbb217736", "external_references": [{"source_name": "Lab52 WIRTE Apr 2019", "description": "S2 Grupo. (2019, April 2). WIRTE Group attacking the Middle East. Retrieved May 24, 2019.", "url": "https://lab52.io/blog/wirte-group-attacking-the-middle-east/"}], "object_marking_refs": ["marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168"], "x_mitre_attack_spec_version": "2.1.0", "x_mitre_modified_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b5

## Match Group -> Relationships Intrusion-set ID
Then, I just take all the group_relationships I got, and look for the specific `intrusion-set (Group)` id in the groups STIX objects. Once there is a match, I create new fields on the `intrusion-set (Group)` STIX object to add additional information about the `attack-pattern (Technique)` in the relationship object. The most important additional metadata is the target_ref field which points to the specific `attack-pattern (Technique)` id involving the `group`. The results are then added to a new list named `group_techniques_ref` .

In [18]:
import json
group_techniques_ref = []
for g in groups:
    for rel in group_relationships:
        if g['id'] == rel['source_ref']:
            gs = json.loads(g.serialize())
            gs
            gs['technique_ref'] = rel['target_ref']
            gs['relationship_description'] = rel['description']
            gs['relationship_id'] = rel['id']
            group_techniques_ref.append(gs)

## Match Attack-patterns -> Intrusion-set object ID
I apply the same concept as before, and just loop through all the attack-pattern objects and look for the specific attack-pattern id in the initial relationships STIX objects. Once there is a match, I add additional information from the attack-pattern itself to the python dictionaries in the `group_techniques_ref` list. The results then get added to a new list named `groups_use_techniques`.

In [19]:
groups_use_techniques = []
for gt in group_techniques_ref:
    for t in techniques:
        if gt['technique_ref'] == t['id']:
            tactic_list = list()
            for phase in t['kill_chain_phases']:
                tactic_list.append(phase['phase_name'])
                gt['technique'] = t['name']
                gt['technique_description'] = t['description']
                gt['tactic'] = tactic_list
                gt['technique_id'] = t['external_references'][0]['external_id']
                gt['matrix'] =  t['external_references'][0]['source_name']
                if 'x_mitre_platforms' in t.keys():
                    gt['platform'] = t['x_mitre_platforms']
                if 'x_mitre_data_sources' in t.keys():
                    gt['data_sources'] = t['x_mitre_data_sources']
                if 'x_mitre_permissions_required' in t.keys():
                    gt['permissions_required'] = t['x_mitre_permissions_required']
                if 'x_mitre_effective_permissions' in t.keys():
                    gt['effective_permissions'] = t['x_mitre_effective_permissions']
                groups_use_techniques.append(gt)
groups_use_techniques[0]

{'type': 'intrusion-set',
 'spec_version': '2.1',
 'id': 'intrusion-set--2e5d3a83-fe00-41a5-9b60-237efc84832f',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'created': '2017-05-31T21:31:46.025Z',
 'modified': '2022-04-25T14:00:00.188Z',
 'name': 'Moafee',
 'description': '[Moafee](https://attack.mitre.org/groups/G0002) is a threat group that appears to operate from the Guandong Province of China. Due to overlapping TTPs, including similar custom tools, Moafee is thought to have a direct or indirect relationship with the threat group [DragonOK](https://attack.mitre.org/groups/G0017). (Citation: Haq 2014)',
 'aliases': ['Moafee'],
 'external_references': [{'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/groups/G0002',
   'external_id': 'G0002'},
  {'source_name': 'Moafee', 'description': '(Citation: Haq 2014)'},
  {'source_name': 'Haq 2014',
   'description': 'Haq, T., Moran, N., Scott, M., & Vashisht, S. O. (2014, September 10). The Path to Ma

# Retrive all Techniques used by all Groups - (Automatic)
-----------------------

In [20]:
from attackcti import attack_client
lift = attack_client()

%time techniques_used = lift.get_techniques_used_by_all_groups()

CPU times: user 3.14 s, sys: 171 ms, total: 3.31 s
Wall time: 29.7 s


In [21]:
len(techniques_used)

4046

In [22]:
techniques_used[0]

{'type': 'intrusion-set',
 'spec_version': '2.1',
 'id': 'intrusion-set--2e5d3a83-fe00-41a5-9b60-237efc84832f',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'created': '2017-05-31T21:31:46.025Z',
 'modified': '2022-04-25T14:00:00.188Z',
 'name': 'Moafee',
 'description': '[Moafee](https://attack.mitre.org/groups/G0002) is a threat group that appears to operate from the Guandong Province of China. Due to overlapping TTPs, including similar custom tools, Moafee is thought to have a direct or indirect relationship with the threat group [DragonOK](https://attack.mitre.org/groups/G0017). (Citation: Haq 2014)',
 'aliases': ['Moafee'],
 'external_references': [{'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/groups/G0002',
   'external_id': 'G0002'},
  {'source_name': 'Moafee', 'description': '(Citation: Haq 2014)'},
  {'source_name': 'Haq 2014',
   'description': 'Haq, T., Moran, N., Scott, M., & Vashisht, S. O. (2014, September 10). The Path to Ma

# Create Navigator Group Layer Files - (Manual)
-----------------------

## Create a list of group dictionaries
To make things easier, I create a list of dictionaries where each group name is the main key and the value is a list where I append every single technique involving that group. I get that information from the `get_techniques_used_by_all_groups()` results.

In [24]:
groups = lift.get_groups()
groups_list = []
for g in groups:
    group_dict = dict()
    group_dict[g['name']] = []
    groups_list.append(group_dict)
groups_list[89]

{'Inception': []}

## Group techniques by group
We can then use the output of the `get_techniques_used_by_all_groups()` function and start appending techniques to the dictionaries with the key name that matches the group name being involved with each technique.

In [25]:
for group in groups_list:
    for group_name,techniques_list in group.items():
        for gut in techniques_used:
            if group_name == gut['name']:
                technique_dict = dict()
                technique_dict['techniqueId'] = gut['technique_id']
                technique_dict['techniqueName'] = gut['technique']
                technique_dict['comment'] = gut['relationship_description']
                technique_dict['tactic'] = gut['tactic']
                technique_dict['group_id'] = gut['external_references'][0]['external_id']
                techniques_list.append(technique_dict)
groups_list[89]

{'Inception': [{'techniqueId': 'T1027.013',
   'techniqueName': 'Encrypted/Encoded File',
   'comment': '[Inception](https://attack.mitre.org/groups/G0100) has encrypted malware payloads dropped on victim machines with AES and RC4 encryption.(Citation: Kaspersky Cloud Atlas December 2014)',
   'tactic': [KillChainPhase(kill_chain_name='mitre-attack', phase_name='defense-evasion')],
   'group_id': 'G0100'},
  {'techniqueId': 'T1547.001',
   'techniqueName': 'Registry Run Keys / Startup Folder',
   'comment': '[Inception](https://attack.mitre.org/groups/G0100) has maintained persistence by modifying Registry run key value \n <code>HKEY_CURRENT_USER\\Software\\Microsoft\\Windows\\CurrentVersion\\Run\\</code>.(Citation: Kaspersky Cloud Atlas December 2014)',
   'tactic': [KillChainPhase(kill_chain_name='mitre-attack', phase_name='persistence'),
    KillChainPhase(kill_chain_name='mitre-attack', phase_name='privilege-escalation')],
   'group_id': 'G0100'},
  {'techniqueId': 'T1588.002',
   

## Run dynamic navigator layer template

In [26]:
import json
for group in groups_list:
    for k,v in group.items():
        if v:
            actor_layer = {
                "description": ("Enterprise techniques used by {0}, ATT&CK group {1} v1.0".format(k,v[0]['group_id'])),
                "name": ("{0} ({1})".format(k,v[0]['group_id'])),
                "domain": "enterprise-attack",
                "versions": {
                    "attack": "16",
                    "navigator": "5.1.0",
                    "layer": "4.5"
                },
                "techniques": [
                    {
                        "score": 1,
                        "techniqueID" : technique['techniqueId'],
                        "techniqueName" : technique['techniqueName'],
                        "comment": technique['comment']
                    } for technique in v
                ],
                "gradient": {
                    "colors": [
                        "#ffffff",
                        "#ff6666"
                    ],
                    "minValue": 0,
                    "maxValue": 1
                },
                "legendItems": [
                    {
                        "label": ("used by {}".format(k)),
                        "color": "#ff6666"
                    }
                ]
            }
            with open(('{0}_{1}.json'.format(k,v[0]['group_id'])), 'w') as f:
                f.write(json.dumps(actor_layer))

In [27]:
! ls *.json

ALLANITE_G1000.json           LAPSUS$_G1004.json
APT-C-23_G1028.json           Lazarus Group_G0032.json
APT-C-36_G0099.json           LazyScripter_G0140.json
APT12_G0005.json              Leafminer_G0077.json
APT16_G0023.json              Leviathan_G0065.json
APT17_G0025.json              LuminousMoth_G1014.json
APT18_G0026.json              Machete_G0095.json
APT19_G0073.json              Magic Hound_G0059.json
APT1_G0006.json               Malteiro_G1026.json
APT28_G0007.json              Metador_G1013.json
APT29_G0016.json              Moafee_G0002.json
APT30_G0013.json              Mofang_G0103.json
APT32_G0050.json              Molerats_G0021.json
APT33_G0064.json              Moonstone Sleet_G1036.json
APT37_G0067.json              Moses Staff_G1009.json
APT38_G0082.json              MoustachedBouncer_G1019.json
APT39_G0087.json              MuddyWater_G0069.json
APT3_G0022.json               Mustang Panda_G0129.json
APT41_G0096.json              Mustard Tempest_G1020.json
APT5_G

We can delete all the files with the following command.

In [28]:
! rm *.json

# Create Navigator Group Layer Files - (Automatic)
-----------------------

In [29]:
from attackcti import attack_client
lift = attack_client()

%time lift.export_groups_navigator_layers()

CPU times: user 3.34 s, sys: 314 ms, total: 3.66 s
Wall time: 49.9 s


In [30]:
! ls *.json

ALLANITE_G1000.json           LAPSUS$_G1004.json
APT-C-23_G1028.json           Lazarus Group_G0032.json
APT-C-36_G0099.json           LazyScripter_G0140.json
APT12_G0005.json              Leafminer_G0077.json
APT16_G0023.json              Leviathan_G0065.json
APT17_G0025.json              LuminousMoth_G1014.json
APT18_G0026.json              Machete_G0095.json
APT19_G0073.json              Magic Hound_G0059.json
APT1_G0006.json               Malteiro_G1026.json
APT28_G0007.json              Metador_G1013.json
APT29_G0016.json              Moafee_G0002.json
APT30_G0013.json              Mofang_G0103.json
APT32_G0050.json              Molerats_G0021.json
APT33_G0064.json              Moonstone Sleet_G1036.json
APT37_G0067.json              Moses Staff_G1009.json
APT38_G0082.json              MoustachedBouncer_G1019.json
APT39_G0087.json              MuddyWater_G0069.json
APT3_G0022.json               Mustang Panda_G0129.json
APT41_G0096.json              Mustard Tempest_G1020.json
APT5_G

We can delete all the files with the following command.

In [31]:
! rm *.json